In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split



In [2]:
dataset = "data.csv"
model_save_path = "main_model.hdf5"

In [3]:
df = pd.read_csv(dataset, index_col=0)
NUM_CLASSES = df["label"].nunique()


# df.replace({"labels": {"up": 1, "down": 2, "left": 3, "right": 4, "forward": 5, "backward": 6}}, inplace=True)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(y)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=333)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((42, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
Total params: 1,158
Trainable params: 1,158
Non-trainable params: 0
_________________________________________________________________


In [7]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_test, y_test), callbacks=[cp_callback, es_callback])

Epoch 1/1000


2023-04-01 19:00:15.780206: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


28/28 [==============================] - ETA: 0s - loss: 2.1339 - accuracy: 0.1455
Epoch 1: saving model to main_model.hdf5
28/28 [==============================] - 2s 24ms/step - loss: 2.1339 - accuracy: 0.1455 - val_loss: 1.8747 - val_accuracy: 0.3730
Epoch 2/1000
24/28 [========================>.....] - ETA: 0s - loss: 1.8556 - accuracy: 0.2777
Epoch 2: saving model to main_model.hdf5
28/28 [==============================] - 0s 9ms/step - loss: 1.8405 - accuracy: 0.2840 - val_loss: 1.6089 - val_accuracy: 0.4194
Epoch 3/1000
24/28 [========================>.....] - ETA: 0s - loss: 1.6445 - accuracy: 0.3623
Epoch 3: saving model to main_model.hdf5
28/28 [==============================] - 0s 9ms/step - loss: 1.6324 - accuracy: 0.3636 - val_loss: 1.3856 - val_accuracy: 0.5215
Epoch 4/1000
24/28 [========================>.....] - ETA: 0s - loss: 1.4616 - accuracy: 0.4440
Epoch 4: saving model to main_model.hdf5
28/28 [==============================] - 0s 10ms/step - loss: 1.4549 - accura

In [10]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 [==============================] - 0s 9ms/step - loss: 0.0325 - accuracy: 0.9916


In [11]:
model = tf.keras.models.load_model(model_save_path)
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))


1/1 [==============================] - 0s 74ms/step
[9.99130428e-01 2.17065844e-05 6.54856965e-04 1.80067815e-04
 1.52482418e-08 1.71926761e-11 1.11696670e-06 1.19279575e-05]
0


In [13]:
print((max(np.squeeze(predict_result))))

0.9991304
